In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
import ML_two_class_for_autogluon_DC

c:\python_3_8_9\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
temperature_bp_raw_data = pd.read_excel("..//raw_data//Pilot_dataset.xlsx", sheet_name = "Temperature _BP")
pressure_raw_data = pd.read_excel("..//raw_data//Pilot_dataset.xlsx", sheet_name = "Pressure")

In [3]:
temperature_bp_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   年齡                69 non-null     int64         
 1   性別                69 non-null     object        
 2   記錄時間              69 non-null     datetime64[ns]
 3   體溫(B.T,(C))       13 non-null     float64       
 4   體重 - 公斤 (每日)      0 non-null      float64       
 5   Art BP Systolic   46 non-null     float64       
 6   Art BP Diastolic  46 non-null     float64       
 7   Art BP Mean       62 non-null     float64       
 8   NBP Systolic      6 non-null      float64       
 9   NBP Diastolic     6 non-null      float64       
 10  NBP Mean          6 non-null      float64       
 11  Heart Rate        57 non-null     float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 6.6+ KB


In [4]:
pressure_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10854 entries, 0 to 10853
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   時間        10854 non-null  datetime64[ns]
 1   Arterial  10848 non-null  float64       
 2   Venous    10848 non-null  float64       
 3   PF        10848 non-null  float64       
 4   PU        10848 non-null  float64       
 5   TMP       10848 non-null  float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 508.9 KB


In [5]:
temperature_bp_raw_data.describe()

,年齡,"體溫(B.T,(C))",體重 - 公斤 (每日),Art BP Systolic,Art BP Diastolic,Art BP Mean,NBP Systolic,NBP Diastolic,NBP Mean,Heart Rate
count,69.0,13.000000,0.0,46.000000,46.000000,62.000000,6.000000,6.000000,6.000000,57.000000
mean,0.0,36.507692,NaN,44.826087,40.913043,42.580645,47.666667,26.833333,33.000000,141.596491
std,0.0,0.253185,NaN,7.732484,8.307496,8.102921,13.321662,10.647378,11.471704,43.336252
min,0.0,36.100000,NaN,32.000000,25.000000,27.000000,26.000000,13.000000,17.000000,0.000000
25%,0.0,36.300000,NaN,39.000000,34.250000,37.000000,40.500000,19.250000,26.000000,152.000000
50%,0.0,36.500000,NaN,43.000000,39.000000,41.000000,53.000000,27.000000,32.500000,155.000000
75%,0.0,36.600000,NaN,47.000000,44.000000,47.500000,56.500000,34.750000,42.750000,157.000000
max,0.0,37.100000,NaN,63.000000,59.000000,62.000000,60.000000,40.000000,46.000000,167.000000


In [6]:
pressure_raw_data.describe()

,Arterial,Venous,PF,PU,TMP
count,10848.000000,10848.000000,10848.000000,10848.000000,10848.000000
mean,6.575037,19.993824,50.942017,-0.711652,35.941095
std,5.352332,13.418114,20.520494,1.459703,16.747396
min,-1.000000,-21.000000,-20.000000,-46.000000,0.000000
25%,2.000000,9.000000,34.000000,-1.000000,22.000000
50%,5.000000,16.000000,46.000000,-1.000000,31.000000
75%,10.000000,29.000000,65.000000,0.000000,47.000000
max,21.000000,52.000000,508.000000,2.000000,285.000000


In [4]:
# 定義是否有低血壓
# 0：無低血壓；1：有低血壓

def identify_hypotension_or_not(BP_mean, NBP_mean):
    """
    低血壓定義：Art BP Mean< 49, 若沒有Art BP Mean則NBP Mean< 49
    """
    if np.isnan(BP_mean) == False:
        if BP_mean < 49:
            return 1
        else:
            return 0
    else:
        if NBP_mean < 49:
            return 1
        else:
            return 0

temperature_bp_raw_data["Hypotension"] = temperature_bp_raw_data.apply(lambda x: identify_hypotension_or_not(BP_mean = x["Art BP Mean"], NBP_mean = x["NBP Mean"]), axis = 1)

In [5]:
# 定義出每筆資料應該要用哪個時間區段當作訓練資料

one_hour_timedelta = timedelta(hours = 1)
temperature_bp_raw_data["end_of_the_time_of_data"] = temperature_bp_raw_data["記錄時間"].apply(lambda x: x - one_hour_timedelta)
temperature_bp_raw_data["start_of_the_time_of_data"] = temperature_bp_raw_data["end_of_the_time_of_data"].apply(lambda x: x - one_hour_timedelta)

In [6]:
def match_data(start_of_the_time, end_of_the_time):
    select_pressure_data = pressure_raw_data[(pressure_raw_data["時間"] >= start_of_the_time) & (pressure_raw_data["時間"] <= end_of_the_time)]
    if select_pressure_data.shape[0] > 0:
        return select_pressure_data.to_dict("list")
    else:
        return np.nan

temperature_bp_raw_data["Match_Pressure"] = temperature_bp_raw_data.apply(lambda x: match_data(start_of_the_time = x["start_of_the_time_of_data"], 
                                                                                                end_of_the_time = x["end_of_the_time_of_data"]), axis = 1)      

# 排除血壓沒有配對到的資料，以及沒有 Outcome 的資料
temperature_bp_raw_data = temperature_bp_raw_data[(temperature_bp_raw_data["Match_Pressure"].isna() == False) & (temperature_bp_raw_data["Art BP Mean"].isna() == False)].reset_index(drop = True)

In [7]:
temperature_bp_raw_data = pd.concat([
    temperature_bp_raw_data, pd.DataFrame(temperature_bp_raw_data["Match_Pressure"].tolist()).drop(columns = ["時間"])
], axis = 1)

In [8]:
temperature_bp_raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   年齡                         60 non-null     int64         
 1   性別                         60 non-null     object        
 2   記錄時間                       60 non-null     datetime64[ns]
 3   體溫(B.T,(C))                13 non-null     float64       
 4   體重 - 公斤 (每日)               0 non-null      float64       
 5   Art BP Systolic            45 non-null     float64       
 6   Art BP Diastolic           45 non-null     float64       
 7   Art BP Mean                60 non-null     float64       
 8   NBP Systolic               5 non-null      float64       
 9   NBP Diastolic              5 non-null      float64       
 10  NBP Mean                   5 non-null      float64       
 11  Heart Rate                 48 non-null     float64       
 12  end_of_the

In [9]:
# 計算四個Pressure 的統計量
Arterial_statistics = temperature_bp_raw_data["Arterial"].apply(lambda x: {"Arterial_Mean": np.mean(x), "Arterial_Std": np.std(x), "Arterial_Min": np.min(x), "Arterial_Max": np.max(x)}).tolist()
Venous_statistics = temperature_bp_raw_data["Venous"].apply(lambda x: {"Venous_Mean": np.mean(x), "Venous_Std": np.std(x), "Venous_Min": np.min(x), "Venous_Max": np.max(x)}).tolist()
PF_statistics = temperature_bp_raw_data["PF"].apply(lambda x: {"PF_Mean": np.mean(x), "PF_Std": np.std(x), "PF_Min": np.min(x), "PF_Max": np.max(x)}).tolist()
TMP_statistics = temperature_bp_raw_data["TMP"].apply(lambda x: {"TMP_Mean": np.mean(x), "TMP_Std": np.std(x), "TMP_Min": np.min(x), "TMP_Max": np.max(x)}).tolist()

In [10]:
# 將記錄時間、四種血壓統計量、心率與是否低血壓合併成一個DataFrame
inputData_and_label = pd.concat([
    temperature_bp_raw_data["記錄時間"],
    pd.DataFrame(Arterial_statistics), 
    pd.DataFrame(Venous_statistics),
    pd.DataFrame(PF_statistics),
    pd.DataFrame(TMP_statistics),
    temperature_bp_raw_data[["Hypotension"]]
], axis = 1).sort_values(["記錄時間"])

In [10]:
inputData_and_label["Hypotension"].value_counts()

1    45
0    15
Name: Hypotension, dtype: int64

In [15]:
inputData_and_label.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60 entries, 0 to 59
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   記錄時間           60 non-null     datetime64[ns]
 1   Arterial_Mean  60 non-null     float64       
 2   Arterial_Std   60 non-null     float64       
 3   Arterial_Min   60 non-null     float64       
 4   Arterial_Max   60 non-null     float64       
 5   Venous_Mean    60 non-null     float64       
 6   Venous_Std     60 non-null     float64       
 7   Venous_Min     60 non-null     float64       
 8   Venous_Max     60 non-null     float64       
 9   PF_Mean        60 non-null     float64       
 10  PF_Std         60 non-null     float64       
 11  PF_Min         60 non-null     float64       
 12  PF_Max         60 non-null     float64       
 13  TMP_Mean       60 non-null     float64       
 14  TMP_Std        60 non-null     float64       
 15  TMP_Min        60 non-nul

In [11]:
# 切割訓練與測試資料
trainData, testData = train_test_split(inputData_and_label.drop(columns = ["記錄時間"]), test_size = 0.2, random_state = 12345)
trainData, valiData = train_test_split(trainData, test_size = 0.25, random_state = 12345)

# 定義出 xtrain, xtest, ytrain, ytest
xtrain, ytrain = trainData.drop(columns = ["Hypotension"]), trainData["Hypotension"]
xvali, yvali = valiData.drop(columns = ["Hypotension"]), valiData["Hypotension"]
xtest, ytest = testData.drop(columns = ["Hypotension"]), testData["Hypotension"]

In [12]:
totalResult = ML_two_class_for_autogluon_DC.model_fit(trainData = pd.concat([xtrain, ytrain], axis = 1), 
                                                    valiData = pd.concat([xvali, yvali], axis = 1), 
                                                    testData = pd.concat([xtest, ytest], axis = 1), 
                                                    input_features = list(xtrain.columns), 
                                                    target_label = "Hypotension", 
                                                    hyperparameter_tuning = "bayesopt", 
                                                    feature_importances = "PermutationImportance")

100%|██████████| 1/1 [00:00<00:00, 13.69it/s]
Traceback (most recent call last):
  File "c:\python_3_8_9\lib\site-packages\autogluon\core\utils\try_import.py", line 30, in try_import_mxnet
    import mxnet as mx
ModuleNotFoundError: No module named 'mxnet'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\python_3_8_9\lib\site-packages\autogluon\tabular\trainer\abstract_trainer.py", line 1090, in _train_single_full
    hpo_models, hpo_model_performances, hpo_results = model.hyperparameter_tune(X=X, y=y, X_val=X_val, y_val=y_val, scheduler_options=hyperparameter_tune_kwargs, **model_fit_kwargs)
  File "c:\python_3_8_9\lib\site-packages\autogluon\core\models\abstract\abstract_model.py", line 701, in hyperparameter_tune
    return self._hyperparameter_tune(scheduler_options=scheduler_options, **kwargs)
  File "c:\python_3_8_9\lib\site-packages\autogluon\tabular\models\tabular_nn\tabular_nn_model.py", line 711, in _hyperpara

Get KNeighborsUnif, train feature importances
Get KNeighborsUnif, vali feature importances


4it [00:00,  8.88it/s]

Get KNeighborsUnif, test feature importances
Get KNeighborsDist, train feature importances


6it [00:00,  9.33it/s]

...

Get LightGBMLarge_FULL, vali feature importances
Get LightGBMLarge_FULL, test feature importances


76it [00:13,  7.46it/s]

Get WeightedEnsemble_L2_FULL, train feature importances
Get WeightedEnsemble_L2_FULL, vali feature importances


78it [00:13,  5.65it/s]

Get WeightedEnsemble_L2_FULL, test feature importances


In [15]:
pd.DataFrame(totalResult[0]).to_excel("..//result//ML-result-20220922.xlsx", index = None)

In [16]:
pd.DataFrame(totalResult[1]).to_excel("..//result//ML-FeatureImportance-20220922.xlsx", index = None)